In [9]:
import numpy as np
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import random
import math
import gym
from gym import spaces
from itertools import cycle
import torch.nn.functional as F
from pytorch_forecasting import TimeSeriesDataSet
import torch
from torch.utils.data import DataLoader

from data.function.load_data import load_data, load_data_long_format
from data.function.rolling_window import rolling_window_datasets
from technical_analysys.add_indicators import add_indicators, add_returns, add_log_returns
from data.edit import normalize_data, standardize_data

ImportError: cannot import name 'load_data_long_format' from 'data.function.load_data' (C:\Users\jmask\PycharmProjects\RL_tester\data\function\load_data.py)

In [7]:
# Set seeds for reproducibility
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [3]:
df = load_data_long_format(['EURUSD', 'USDJPY', 'EURJPY'], '1D')


100%|██████████| 2/2 [00:00<00:00, 499.95it/s]


In [4]:
df_train

,Open,High,Low,Close,Open,High,Low,Close,Open,High,Low,Close,Even_Odd_Month,Even_Odd_Day,Even_Odd_Year,RSI_14,ATR_24
Currency,EURUSD,EURUSD,EURUSD,EURUSD,USDJPY,USDJPY,USDJPY,USDJPY,EURJPY,EURJPY,EURJPY,EURJPY,EURUSD,USDJPY,EURJPY,EURUSD,EURUSD
Date,,,,,,,,,,,,,,,,,
2003-01-02,1.04750,1.04880,1.03380,1.03840,118.970,120.300,118.790,119.890,124.610,124.880,124.130,124.470,1,0,1,0.000000,0.009300
2003-01-03,1.03830,1.04370,1.03420,1.04230,119.900,120.230,119.620,119.710,124.480,125.010,124.270,124.750,1,1,1,0.297710,0.009367
2003-01-05,1.04200,1.04510,1.04180,1.04480,119.900,119.900,119.180,119.230,124.910,124.970,124.440,124.550,1,1,1,0.410256,0.007850
2003-01-06,1.04470,1.04960,1.04250,1.04400,119.240,119.660,118.580,119.540,124.540,124.960,124.290,124.760,1,0,1,0.390244,0.007700
2003-01-07,1.04390,1.04530,1.03980,1.04210,119.550,120.630,119.370,120.190,124.770,125.520,124.580,125.240,1,1,1,0.349727,0.007333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,1.06264,1.06604,1.06151,1.06454,132.527,133.190,132.477,132.888,140.847,141.710,140.776,141.468,0,0,0,0.662479,0.008475
2022-12-27,1.06454,1.06694,1.06113,1.06415,132.887,134.400,132.736,133.971,141.469,142.936,141.390,142.563,0,1,0,0.685377,0.008407


In [5]:
max_prediction_length = 1  # Predict one step at a time
max_encoder_length = 12
training_cutoff = df_train["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    df_train[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="target",  # Target column to predict
    group_ids=["group_id"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=[],
    static_reals=[],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["target"],
)

KeyError: 'time_idx'